## Assignment 2
### Speech & NLP

### Name : Prerit Gupta, Roll No: 14EC35001

### Task 1
#### Generative Models

(a) Naive’s Bayes Classifier: The NB Classifier is based on the Baye’s theorum : p(y|x) = p(x|y)p(y)/p(x)  = p(x,y)/p(x), but it doesn’t model the conditional probability directly. It models the joint probability, and after that it calculates p(y|x).  This model can be used to generate data on the basis of p(x,y) modelled.

(b) Gaussian Mixture Model: It represents the mixture distributions or joint distributions on gaussian distributed assumption of variables. A GMM can be used to generate data from the joint gaussian distribution of variable modelled.

(c) GANs (Generative Adversarial Networks) : The generator network of the GAN models the underlying distribution of the supplied real data and becomes profidient in learning the true distribution of the real dataset by fooling the distinguishing power of the discriminator. Hence, GAN is a generative model used to generate distribution of a given dataset.
 
(d) LDA(Latent Dirichlet Allocation): It is a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar.  

#### Discriminative Models
(a) Neural Networks: In neural networks, the likelihood and log-likelihood objective functions are both equivalent to the probability distribution p(y|x) as follows: L(θ) = L(θ;X,y) = L(y|X,θ) where the model θ is chosen to make the p(y|x) as high probability as possible. The task objective is classification of input data which is discrimination among possible class outcomes.

(b) Logistic Regression: Logistic regression measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities using a logisitic function (cumulative logical distribution). Logistic regression predicts the probability of particular outcomes rather than the outcomes themselves.

(c) SVM (Support Vector Machine): Discriminative classifiers model the posterior (i.e. probability of class  given data, Pr(y|x)) directly from the data. SVM is discriminative because it  explicitly learns the class boundary between the two classes.

(d) Decision Tree : A decision tree uses a tree-like graph to perform classification. It is discriminative as it distinguishes the class label of sampe conditioned on its underlying feature attributes.

### Task 2

In [2]:
from nltk.corpus import treebank,brown

corpus = brown.tagged_sents(tagset='universal')[:-100] 

tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0].lower()
        tag= elem[1]

        if w not in word_dict:
            word_dict[w]=0

        if tag not in tag_dict:
            tag_dict[tag]=0

        word_dict[w]+=1
        tag_dict[tag]+=1

print(len(word_dict))
print(len(tag_dict))
        
test_data= brown.tagged_sents(tagset='universal')[-10:]

print(len(test_data))

49680
12
10


#### Building the transition, start and emission matrices from the training data 

In [33]:
import numpy as np
from nltk import ngrams

state_len = len(tag_dict)
word_len = len(word_dict)
state = {}
state_val = {}
i = 0
for s in tag_dict.keys():
    state[s] = i
    state_val[i] = s
    i = i+1
word = {}
i = 0
for w in word_dict.keys():
    word[w] = i
    i = i+1 
P = np.zeros((state_len,state_len))
S = np.zeros(state_len)
O = np.zeros((state_len,word_len))
for sent in corpus:
    S[state[sent[0][1]]]+=1
    for elem in sent:
        O[state[elem[1]],word[elem[0].lower()]]+=1
    for elem1,elem2 in ngrams(sent,2, pad_left=False,pad_right=False):
        P[state[elem1[1]],state[elem2[1]]]+=1
        
S = [s/S.sum() for s in S]
P_sum = np.sum(P,axis=1)
for i,p in enumerate(P):
    P[i] = p[:]/P_sum[i]
    
#Laplacian Smoothing in Emission Matrix
k = 0.001
O_sum = np.sum(O,axis=1)
for i,o in enumerate(O):
    for j,val in enumerate(o):
        O[i,j] = (val+k)/(O_sum[i]+k*word_len) 
        
#Laplacian Smoothing for unknown words
O_unk = np.zeros(state_len)
for i in range(state_len):
    O_unk[i] = k/(O_sum[i]+k*word_len)

#### Viterbi Algorithm

In [34]:
def viterbi_algo(sequence):
    M = len(sequence)
    
    T = np.zeros((state_len,M))
    for i in range(state_len):
        T[i][0] = S[i]*O[i,word[sequence[0]]]
    for j in range(1,M):
        for i in range(state_len):
            for k in range(state_len):
                if (sequence[j] in word.keys()):
                    T[i][j] += T[k][j-1]*P[k,i]*O[i,word[sequence[j]]]
                else:
                    T[i][j] += T[k][j-1]*P[k,i]*O_unk[i]
    states = np.zeros(M)
    states[-1] = np.argmax(T[:,-1])
    probs = np.zeros(state_len)
    for j in reversed(range(M-1)):
        for i in range(state_len):
            if (sequence[j+1] in word.keys()):
                probs[i] = T[i,j]*P[i,int(states[j+1])]*O[int(states[j+1]),word[sequence[j+1]]]
            else:
                probs[i] = T[i,j]*P[i,int(states[j+1])]*O_unk[int(states[j+1])]
        states[j] = np.argmax(probs)

    return (states,T[int(states[-1])][-1]) 

#### Evaluation on test sentences

In [50]:
sequences = [[ele[0].lower() for ele in sent] for sent in test_data]
labels = [[ele[1] for ele in sent] for sent in test_data]
import time
t_acc = 0
t_len = 0
for i,sequence in enumerate(sequences):
    
    t=time.time()
    best=viterbi_algo(sequence)
    t2=time.time()-t
    acc=0
    for j,k in enumerate(best[0]):
        if(state_val[int(k)]==labels[i][j]):
            acc+=1
    t_acc += acc
    t_len +=len(sequence)
    acc/=len(sequence)
    print('Sentence: "'+' '.join(s for s in sequence)+ '"')
    print ('Length '+ str(len(sequence)))
    print('The sequence '+ ','.join([state_val[int(k)] for k in best[0]])+ ' gave the best score of '+ str(best[1]))
    print ('True sequence:'+','.join(l for l in labels[i]))
    print ('Accuracy: '+str(round(acc*100,3))+'%'+' Time taken '+ str(round(t2,3))+'s' )   
    print('\n')
t_acc /=t_len

Sentence: "you can't very well sidle up to people on the street and ask if they want to buy a hot bodhisattva ."
Length 22
The sequence PRON,VERB,ADV,ADV,VERB,PRT,ADP,NOUN,ADP,DET,NOUN,CONJ,VERB,ADP,PRON,VERB,PRT,VERB,DET,ADJ,NOUN,. gave the best score of 5.273308041509585e-66
True sequence:PRON,VERB,ADV,ADV,VERB,ADP,ADP,NOUN,ADP,DET,NOUN,CONJ,VERB,ADP,PRON,VERB,PRT,VERB,DET,ADJ,NOUN,.
Accuracy: 95.455% Time taken 0.044s


Sentence: "additionally , since you're going to be hors de combat pretty soon with sprue , yaws , delhi boil , the granville wilt , liver fluke , bilharziasis , and a host of other complications of the hex you've aroused , you mustn't expect to be lionized socially ."
Length 49
The sequence ADV,.,ADP,PRT,VERB,PRT,VERB,X,X,VERB,ADV,ADV,ADP,NOUN,.,X,.,NOUN,NOUN,.,DET,NOUN,VERB,.,NOUN,.,.,X,.,CONJ,DET,NOUN,ADP,ADJ,NOUN,ADP,DET,NOUN,PRT,VERB,.,PRON,VERB,VERB,PRT,VERB,VERB,ADV,. gave the best score of 1.5861544923363735e-172
True sequence:ADV,.,ADP,PRT,VERB,PRT,VERB,X,X,X

#### Reporting overall accuracy of model

In [52]:
print ("Overall accuracy of HMM model is "+str(round(t_acc*100,3))+"%")

Overall accuracy of HMM model is 93.305%


### Task 3 : CRF

#### Features included for CRF:
(a) Lower Case word (previous,current,next): The lower case form of the word can be used to capture semantics where     same words having phrasal relationship is occuring important for semantics of POS tags.

(b) Binary feature for Title (previous,current,next): Some words have starting letter capital. This can be used to     distinguish nouns like names of places, people, etc and starting of sentence from other POS tags.

(c) Suffix (current):

    (i) 3 characters ending : Marking gerund form of word specifically end with 'ing', also modelling specific           suffixes like 'ant','ist','ive'
    
    (ii) 2 characters ending : Marking plural form of words ending with 'es', also modelling specific suffixes like       'ty','er','al'
    
(d) POS Tags (previous,current,next) : The POS Tags directly model the labels which is are task on the test             sentences.

    
Including the above features is leading to 100% accuracy on the test sentences.

In [3]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

train_sents= corpus

def word2features(sent,i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features ={
    'bias': 1.0,
    'word.lower()': word.lower(),
    'word[-3:]' : word[-3:],
    'word[-2:]' : word[-2:],
    'word.istitle()': word.istitle(),
    'postag' : postag
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True
                
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for i,label in sent]

In [4]:
X_train=[sent2features(s) for s in train_sents]
y_train=[sent2labels(s) for s in train_sents]

X_test=[sent2features(s) for s in test_data]
y_test=[sent2labels(s) for s in test_data]

In [5]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [6]:
y_pred = crf.predict(X_test)
labels=list(crf.classes_)

metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

/home/prerit/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/prerit/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


1.0

In [7]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

          .      1.000     1.000     1.000        33
          X      1.000     1.000     1.000         3
        ADJ      1.000     1.000     1.000        18
        ADP      1.000     1.000     1.000        27
        ADV      1.000     1.000     1.000         9
       VERB      1.000     1.000     1.000        35
        DET      1.000     1.000     1.000        33
       CONJ      1.000     1.000     1.000         7
       NOUN      1.000     1.000     1.000        51
       PRON      1.000     1.000     1.000        12
        PRT      1.000     1.000     1.000        11
        NUM      0.000     0.000     0.000         0

avg / total      1.000     1.000     1.000       239



/home/prerit/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/prerit/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
